# OAO Reader - Motion UBX

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os
import sys

from datetime import datetime, timezone

import numpy as np

import unittest

from file_reader import FileReader

## Main Class

In [2]:
class OaoReader(FileReader):
    '''OAO file - Motion UBX'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)


    def load(self):
        '''Load file into memory'''

        with open(self.filename, "rb") as f:
            self.buffer = f.read()

            if len(self.buffer) > 0:
                self.readHeader()
                self.readData()


    def readHeader(self):
        '''Load OAO header into memory'''

        pass


    def readData(self):
        '''Read GNSS frames into memory'''

        fileSize = len(self.buffer)

        # Lean buffer will only contain the geodetic navigation data, minus start / end tokens and checksums
        leanBuffer = bytearray()

        while self.bufferPtr < fileSize:
            message = self.readFrame(leanBuffer)
                       
        if self.bufferPtr != fileSize:
            raise ValueError('File appears to be truncated')

        dtype = self.getDatatype()
        
        self.rawData = np.frombuffer(leanBuffer, dtype=dtype, count=self.numRecords)
        
        self.consumeRawData()


    def readFrame(self, leanBuffer):
        '''Read OAO frame from buffer'''

        offset = self.bufferPtr
        mode = self.readUnsigned16LE()
        expectedChecksum = self.readUnsigned16LE()

        if mode == 0x0AD0:
            length = 512
        elif mode == 0x0AD1:
            length = 12
        elif mode == 0x0AD2 or mode == 0x0AD3:
            length = 34
        elif mode == 0x0AD4 or mode == 0x0AD5:
            length = 52
        else:
            raise ValueError('Unexpected mode 0x{:x} - offset 0x{:08x}'.format(mode, offset))

        # Main frame content
        payload = self.readBytes(length - 4)

        # Evaluate checksum
        actualChecksum = self.calculateChecksum(mode, payload)
        if actualChecksum != expectedChecksum:
            raise ValueError('Checksum difference - {} vs {}'.format(actualChecksum, expectedChecksum))

        # Append GNSS frame to lean buffer
        if mode == 0x0AD5:
            leanBuffer.extend(payload)
            self.numRecords += 1

        return payload


    def calculateChecksum(self, mode, payload):
        '''Calculate Fletcher's checksum for the OAO frame'''

        checksum_a = checksum_b = 0
        
        # Sum the two first bytes
        for i in range(2):
            checksum_a += mode
            checksum_b += checksum_a
            mode >>= 8

        # Sum the payload bytes
        for byte in payload:
            checksum_a += byte
            checksum_b += checksum_a

        # Bit masking has been deferred until the end
        return (checksum_b & 0xff) << 8 | checksum_a & 0xff


    def getDatatype(self):
        '''Get the datatype of the GNSS frames'''

        # Standard 32-byte GNSS frame - 48 bytes excluding the mode and checksum
        dtype = [
            ('latitude', '<i4'),
            ('longitude', '<i4'),
            ('altitude_msl', '<i4'),
            ('speed', '<u4'),
            ('course', '<u4'),
            ('utc_time', '<u8'),
            ('fix', 'u1'),
            ('satellites', 'u1'),
            ('speed_accuracy', '<u4'),
            ('horizontal_accuracy', '<u4'),
            ('vertical_accuracy', '<u4'),
            ('heading_accuracy', '<u4'),
            ('hdop', '<u2')
        ]

        return np.dtype(dtype)


    def consumeRawData(self):
        '''Consume data - convert to standard data structure'''

        # Convert latitude and longitude to decimal
        self.data['lat'] = self.rawData['latitude'] / 10000000
        self.data['lon'] = self.rawData['longitude'] / 10000000

        # Convert elevation from cm to m
        self.data['ele'] = self.rawData['altitude_msl'] / 1000

        # Convert SOG from mm/s to m/s
        self.data['sog'] = self.rawData['speed'] / 1000

        # Convert COG from integer to decimal
        self.data['cog'] = self.rawData['course'] / 100000

        # Convert timestamp from ms to s
        self.data['timestamp'] = self.rawData['utc_time'] / 1000

        # Copy fix and satellites
        self.data['fix'] = self.rawData['fix']
        self.data['sat'] = self.rawData['satellites']

        # Convert sAcc from mm/s to m/s
        self.data['sacc'] = self.rawData['speed_accuracy'] / 1000

        # Convert hAcc, vAcc and cAcc from mm to m
        self.data['hacc'] = self.rawData['horizontal_accuracy'] / 1000
        self.data['vacc'] = self.rawData['vertical_accuracy'] / 1000
        self.data['cacc'] = self.rawData['heading_accuracy'] / 100000

        # Convert HDOP from integer to decimal
        self.data['hdop'] = self.rawData['hdop'] / 100

## Unit Tests

In [3]:
class TestData(unittest.TestCase):
    '''Class to test data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(oaoReader.numRecords, 43182)

        for fieldName in oaoReader.data:
            self.assertEqual(oaoReader.data[fieldName].size, 43182)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        self.assertEqual(oaoReader.data['hdop'].min(), 0.62)
        self.assertEqual(oaoReader.data['hdop'].max(), 99.99)


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(oaoReader.data['sat'].min(), 0)
        self.assertEqual(oaoReader.data['sat'].max(), 18)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(oaoReader.data['timestamp'].min(), 1649672228.1)
        self.assertEqual(oaoReader.data['timestamp'].max(), 1649677025.9)


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(oaoReader.data['lat'].min(), 50.5710305)
        self.assertEqual(oaoReader.data['lat'].max(), 50.5833157)


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(oaoReader.data['lon'].min(), -2.462027)
        self.assertEqual(oaoReader.data['lon'].max(), -2.4563075)


    def testEle(self):
        '''Test the elevation is as expected'''

        self.assertEqual(oaoReader.data['ele'].min(), -2.899 )
        self.assertEqual(oaoReader.data['ele'].max(), 13.548)


    def testSog(self):
        '''Test the speed over ground is as expected'''

        self.assertEqual(oaoReader.data['sog'].min(), 0)
        self.assertEqual(oaoReader.data['sog'].max(), 17.142)


    def testCog(self):
        '''Test the course over ground is as expected'''

        self.assertEqual(oaoReader.data['cog'].min(), 0.00567)
        self.assertEqual(oaoReader.data['cog'].max(), 359.99618)


    def testFix(self):
        '''Test the fix is as expected'''

        self.assertEqual(oaoReader.data['fix'].min(), 0)
        self.assertEqual(oaoReader.data['fix'].max(), 3)


    def testSacc(self):
        '''Test the speed accuracy is as expected'''

        self.assertEqual(oaoReader.data['sacc'].min(), 0.092)
        self.assertEqual(oaoReader.data['sacc'].max(), 2.118)


    def testHacc(self):
        '''Test the horizontal accuracy is as expected'''

        self.assertEqual(oaoReader.data['hacc'].min(), 0.395)
        self.assertEqual(oaoReader.data['hacc'].max(), 12.051)


    def testVacc(self):
        '''Test the vertical accuracy is as expected'''

        self.assertEqual(oaoReader.data['vacc'].min(), 0.524)
        self.assertEqual(oaoReader.data['vacc'].max(), 11.455)


    def testCacc(self):
        '''Test the course accuracy is as expected'''

        self.assertEqual(oaoReader.data['cacc'].min(), 0.40106)
        self.assertEqual(oaoReader.data['cacc'].max(), 45.01615)

In [4]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20220411', 'Motion_Mini_10Hz_0470_2022-04-11-1117.oao')
    oaoReader = OaoReader(filename)
    oaoReader.load()

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)
    
    print('All done!')

..............

All done!



----------------------------------------------------------------------
Ran 14 tests in 0.009s

OK
